<a href="https://colab.research.google.com/github/paxcalpt/GoogleColab/blob/master/eLife_AuthorDomainStats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hi, Ricardo Henriques over here, I made this bit of code to pull the email domain from the corresponding authors in the eLife papers database. This was mostly out of personal curiosity. Please consider this code wildly untested or validated.

Feel free to use the code as you think best...

In [1]:
!pip install termplotlib
!pip install maxminddb-geolite2
!git clone https://github.com/elifesciences/elife-article-xml.git
domains = {}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.1 MB 2.3 MB/s 
     |████████████████████████████████| 330 kB 36.2 MB/s 
  Created wheel for maxminddb-geolite2: filename=maxminddb_geolite2-2018.703-py3-none-any.whl size=26407052 sha256=0abfc2b5f6d0f83bd04326c650a1f89fcaf8620bfb800e25141a936dbdf1a776
  Stored in directory: /root/.cache/pip/wheels/21/f8/f5/c3d091f638042b802aa461ae6087b6f4106df7bf9a46ad4114
  Created wheel for maxminddb: filename=maxminddb-2.2.0-py2.py3-none-any.whl size=16356 sha256=94b41dacd0ed1433c60d85e03e0e210fab94bfebddc9a8e72f4e7531cac0217d
  Stored in directory: /root/.cache/pip/wheels/3b/eb/d9/b4407ec5d0ccddce61c8d2f4db72a68a77a04adea2245b98ef
Successfully built maxminddb-geolite2 maxminddb
Cloning into 'elife-article-xml'...
remote: Enumerating objects: 120788, done.
remot

In [14]:
import glob, termplotlib, socket
from geolite2 import geolite2

greader = geolite2.reader()  
files_xml = glob.glob("elife-article-xml/articles/*v1.xml")
#print(files_xml)
emailServers = {}
countries = {}

doCountries = False # Change this to False if you want faster calculation
consolidateDomains = True # Change this to consolidate domains (e.g. aaa.bbb.ccc.ddd.eee -> ccc.ddd.eee)

counter = 1
nFiles = len(files_xml)
for fname in files_xml:
  if counter%100==0:
    print(f"Processing {counter}/{nFiles} ({round(100.*counter/nFiles)}%): {fname}")
  counter += 1
  with open(fname) as f:
    txt = f.read()
    #print(txt)
    #break

    ref = 0
    servers = []

    while 1:
      try:
        first = "<email>"
        last = "</email>"
        s = txt[ref:]
        start = s.index(first) + len(first)
        end = s.index(last, start)
        server = s[start:end].split("@")[1]
        ref = ref+end

        if server == "gmail.com":
          continue

        # workout country
        if doCountries:
          if server in domains:
            country = domains[server]
          else:
            #try:
            try:
              ip = socket.gethostbyname(server)
              v = greader.get(ip)
              country = v['country']['iso_code']
              domains[server] = country
              print(v)
            except Exception: 
              country = "Unknown"   

          if country in countries:
            countries[country] += 1
          else:
            countries[country] = 1

        servers.append(server)
        # also consolidate if needed
        if consolidateDomains:
          serverByLevel = server.split(".")
          levels = len(serverByLevel)
          for l in range(3, levels): # consolidated to level 3 if enabled
            serverForLevel = ".".join(serverByLevel[-l:])
            servers.append(serverForLevel)
          
      except ValueError: 
        break
    
    for server in servers:

      if server in emailServers:
        emailServers[server] += 1
      else:
        emailServers[server] = 1

print("\nTop domains:")
keys_server = sorted(emailServers.keys(), key=lambda k: emailServers[k], reverse=True)
for key in keys_server[:20]:
  print(f"{key}: {emailServers[key]}")

print("\nTop countries:")
keys_country = sorted(countries.keys(), key=lambda k: countries[k], reverse=True)
for key in keys_country[:20]:
  print(f"{key}: {countries[key]}")





Processing 100/13644 (1%): elife-article-xml/articles/elife-27563-v1.xml
Processing 200/13644 (1%): elife-article-xml/articles/elife-74314-v1.xml
Processing 300/13644 (2%): elife-article-xml/articles/elife-21137-v1.xml
Processing 400/13644 (3%): elife-article-xml/articles/elife-07847-v1.xml
Processing 500/13644 (4%): elife-article-xml/articles/elife-72884-v1.xml
Processing 600/13644 (4%): elife-article-xml/articles/elife-25281-v1.xml
Processing 700/13644 (5%): elife-article-xml/articles/elife-14845-v1.xml
Processing 800/13644 (6%): elife-article-xml/articles/elife-66155-v1.xml
Processing 900/13644 (7%): elife-article-xml/articles/elife-07383-v1.xml
Processing 1000/13644 (7%): elife-article-xml/articles/elife-03568-v1.xml
Processing 1100/13644 (8%): elife-article-xml/articles/elife-55013-v1.xml
Processing 1200/13644 (9%): elife-article-xml/articles/elife-39383-v1.xml
Processing 1300/13644 (10%): elife-article-xml/articles/elife-04525-v1.xml
Processing 1400/13644 (10%): elife-article-xml

In [15]:
import numpy as np
fig = termplotlib.figure()

count = np.asarray([emailServers[key] for key in keys_server])
keys_server = list(keys_server)
fig.barh(count, keys_server)
fig.show()

cam.ac.uk                         [349]  ████████████████████████████████████████
stanford.edu                      [312]  ███████████████████████████████████▊
ox.ac.uk                          [296]  █████████████████████████████████▉
berkeley.edu                      [269]  ██████████████████████████████▉
ucsf.edu                          [247]  ████████████████████████████▎
ucl.ac.uk                         [204]  ███████████████████████▍
yale.edu                          [194]  ██████████████████████▎
utsouthwestern.edu                [173]  ███████████████████▉
hms.harvard.edu                   [164]  ██████████████████▊
ucsd.edu                          [159]  ██████████████████▎
mit.edu                           [148]  █████████████████
nih.gov                           [146]  ████████████████▊
jhmi.edu                          [141]  ████████████████▏
umich.edu                         [137]  ███████████████▊
uw.edu                            [132]  ███████████████▏
wustl.edu   